## Zeitfenster mit Generatoren anlernen
Beispieldaten: Jena Temperaturen, Kapitel 5.6.2

### Vorbereitung: Daten laden und Datum parsen

In [1]:
import pandas as pd
from math import ceil

pd.set_option('display.max_columns', 6)

data_url=r'https://github.com/tplusone/hanser_ml_zeitreihen/blob/master/Daten/jena_climate_complete_hourly.csv?raw=true'
df = pd.read_csv(data_url)

### Datum parsen und als Index setzen
df['Date Time'] = pd.to_datetime(df['Date Time'])
df = df.set_index('Date Time')

### Daten zeigen
df.head().round(3)

,p (mbar),T (degC),Tpot (K),...,wv (m/s),max. wv (m/s),wd (deg)
Date Time,,,,,,,
2009-01-01 00:00:00,996.528,-8.304,265.118,...,0.520,1.002,174.460
2009-01-01 01:00:00,996.525,-8.065,265.362,...,0.317,0.712,172.417
2009-01-01 02:00:00,996.745,-8.763,264.645,...,0.248,0.607,196.817
2009-01-01 03:00:00,996.987,-8.897,264.492,...,0.177,0.607,157.083
2009-01-01 04:00:00,997.158,-9.348,264.027,...,0.290,0.670,150.093


### a) x- und y-Daten extrahieren und in Trainings-/Testpartitionen einteilen

In [2]:
X = df.values
y = df[['T (degC)']].values
X.shape, y.shape

## 2. Schritt: Train/Test-Split
train_end = ceil(len(X) * 0.8)
X_train = X[:train_end]
y_train = y[:train_end]

X_test = X[train_end:]
y_test = y[train_end:]

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((56104, 14), (14025, 14), (56104, 1), (14025, 1))

### b) Standardisierung der Daten

In [3]:
import joblib
from sklearn.preprocessing import StandardScaler

scaler_x = StandardScaler()
scaler_x.fit(X_train)
X_train = scaler_x.transform(X_train)
X_test = scaler_x.transform(X_test)

scaler_y = StandardScaler()
scaler_y.fit(y_train)
y_train = scaler_y.transform(y_train)
y_test = scaler_y.transform(y_test)

joblib.dump(scaler_x, 'scaler_x.pkl')
joblib.dump(scaler_y, 'scaler_y.pkl')

['scaler_y.pkl']

### c) Daten-Generator zur Organisation des Anlernprozesses

In [4]:
import random
import numpy as np

def data_generator(X, y, window=144, horizon=24, batch_size=1, 
                   epochs=10):
    for epoch in range(epochs):
        X_temp = []
        y_temp = []
        batch_counter = 0
        last_val = len(X) - (window + horizon)
        index_range = list(range(last_val))
        random.shuffle(index_range)
        for idx in index_range:
            X_idx = X[idx:idx+window]
            y_idx = y[idx+window+horizon]
            X_temp.append(X_idx)
            y_temp.append(y_idx)
            batch_counter += 1
            if (batch_counter == batch_size or 
                index_range[-1] == idx):
                yield np.array(X_temp), np.array(y_temp).reshape(-1)
                X_temp = []
                y_temp = []
                batch_counter = 0

### Generator testen

In [5]:
test_gen = data_generator(X_train, y_train)
x_, y_ = next(test_gen)
print(x_.shape, y_.shape)

(1, 144, 14) (1,)


### d) Aufsetzen und Anlernen des Keras-Modells

#### Modellarchitektur festlegen

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Dense, Dropout, Conv1D, 
                                    LSTM, Bidirectional)
from tensorflow.keras.regularizers import l2
reg = l2(0.0001)
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=6, strides=2, 
                input_shape=(144, 14), 
                activation='relu'))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(units=32, dropout=.3,
            kernel_regularizer=reg), input_shape=(144, 14)))            
model.add(Dense(units=1, kernel_regularizer=reg))
model.compile(loss='mse', optimizer='adam', metrics=['mae'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 70, 32)            2720      
_________________________________________________________________
dropout (Dropout)            (None, 70, 32)            0         
_________________________________________________________________
bidirectional (Bidirectional (None, 64)                16640     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 19,425
Trainable params: 19,425
Non-trainable params: 0
_________________________________________________________________


#### Generatoren und Callbacks definieren

In [7]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

batch_size = 128
epochs = 50
window = 144
future = 24
gen_train = data_generator(X_train, y_train, 
                          batch_size=batch_size,
                          epochs=epochs)
gen_test = data_generator(X_test, y_test, 
                          batch_size=batch_size,
                          epochs=epochs)

early = EarlyStopping(monitor='val_loss', patience=2)
check = ModelCheckpoint(filepath='climate_model.h5', 
                        monitor='val_loss', save_best_only=True)

steps_train = ceil(len(X_train-(window+future))/batch_size)
steps_test = ceil(len(X_test-(window+future))/batch_size)

#### Anlernprozess starten

In [8]:
history = model.fit(gen_train, epochs=epochs, 
             steps_per_epoch=steps_train, 
             validation_data=(gen_test), 
             validation_steps=steps_test,
             callbacks=[early, check])

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 439 steps, validate for 110 steps
Epoch 1/50
439/439 [==============================] - 75s 172ms/step - loss: 0.4817 - mae: 0.5100 - val_loss: 0.4053 - val_mae: 0.4732
Epoch 2/50
439/439 [==============================] - 67s 152ms/step - loss: 0.3963 - mae: 0.4622 - val_loss: 0.3964 - val_mae: 0.4736
Epoch 3/50
439/439 [==============================] - 66s 150ms/step - loss: 0.3496 - mae: 0.4343 - val_loss: 0.4307 - val_mae: 0.5082
Epoch 4/50
439/439 [==============================] - 65s 149ms/step - loss: 0.3216 - mae: 0.4158 - val_loss: 0.4240 - val_mae: 0.5022


### e) Evaluation

In [11]:
def predictions_vs_true(X_pred, y_true, model, scaler,
                        window=144, horizon=24):
    X_temp = []
    y_temp = []
    y_temp_simple = []
    num_pred = len(X_pred)-(window+horizon)
    for idx in range(num_pred):
        X_idx = X_pred[idx:idx+window]
        y_idx = y_true[idx+window+horizon]
        X_temp.append(X_idx)
        y_temp.append(y_idx)
        y_temp_simple.append(y_true[idx+window])
    X_temp = np.array(X_temp)
    y_pred = model.predict(X_temp)
    y_pred = scaler.inverse_transform(y_pred)
    y_temp = scaler.inverse_transform(y_temp)
    y_temp_simple = scaler.inverse_transform(y_temp_simple)
    return y_temp, y_pred, y_temp_simple

In [12]:
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_absolute_error

model = load_model('climate_model.h5')
y_true, y_pred, y_temp_simple = predictions_vs_true(X_test, 
                                    y_test, 
                                    model=model, 
                                    scaler=scaler_y)
                                     
print('mae, predictions', mean_absolute_error(y_true, y_pred))
print('mae, simple pred', mean_absolute_error(y_true, y_temp_simple))

mae, predictions 4.0222062736252475
mae, simple pred 4.209516369584565
